<a href="https://colab.research.google.com/github/vir-k01/StyleGAN2-ADA-MicrostructureGeneration/blob/main/StyleGAN2_ADA_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA Training

Code accompanying the manuscript: "Quantification of similarity and physical awareness of microstructures generated via Generative Models" by Sanket Thakre, Vir Karan, Anand Krishna Kanjarla.

-Notebook authored by Vir Karan

## The basic idea

This notebook contains the code needed to prepare a custom dataset and train a StyleGAN2-ADA model in the Google Colab environment. This notebook clones the official repository by Nvidia (https://github.com/NVlabs/stylegan2-ada) and then calls the scripts from that repo. Relevant changes can be made to run this locally or on a workstation. 



(Some aspects of this notebook have been adapted from https://github.com/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb)

**StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.**

In [ ]:
%tensorflow_version 1.x

Make sure that a GPU runtime has been selected. GANs require **a lot** of compute to train, so make sure you've got a GPU with atleast 12GB RAM here. In the case of Colab, the default K80 GPU should suffice for smaller datasets, but it is preferable to use a T4 or P100 or better GPU. 

In [ ]:
!nvidia-smi

Since Colab runtimes are very unreliable wrt stability while training, it is always better to mount Google Drive and save/load all files from there. In this case, we'll also install the StyleGAN repo to drive directly to save time for future runs.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

## Dataset preparation

The StyleGAN requires the dataset to be in `.tfrecords` files. Also, the actual images needed to train the model have to be in the RGB format (to be able to use pre-trained models). The dataset (attached with this repo: https://drive.google.com/drive/folders/1Oz2NlycaIfK0-mebNgz5HTTW4_UuVikQ?usp=sharing) has to first be extracted and converted to this format.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Note: The functions in the next cell are general functions written to extract the microstructural data from the .txt files and save them as np arrays. There are additional arguements present to perform image augmentation (Shifting, Flipping, Rotating, Cropping) as well, however, for this particular work this augmentation is not needed. 

In [ ]:
def read_store_data(filepath):
  readfile = open(filepath, "r")
  arr1 =[]
  for line in readfile:
    Type = line.split(" ")
    arr1.append(Type)
  arr1 = np.asarray(arr1)
  arr1 = arr1.astype(np.float)
  arr1 = np.squeeze(arr1)
  arr1 = arr1[:, 3][:256*256]
  arr1 = np.rot90(np.reshape(arr1, (256,256)))
  plt.imshow(arr1, cmap = 'gray')
  return arr1

def store_array(im, filename):
  fil = open(str(filename) + '.txt', 'w')
  row, column = im.shape
  for y in range(column):
      for x in range(row):
          pixel = im[y, x]
          fil.write(str(x+1) + " " + str(y+1) + " " + str(pixel) + '\n')
  fil.close()

def read_many_disk(Shift, Flip, Rotate, Crop, num_classes, num_images = 50, Angle = 90, width = 128, height =128):
    """ Reads image from disk.
        Parameters:
        ---------------
        Shift        Boolean          Whether to shift the position of the image to the left or right 
        Flip         Boolean          Whether to flip the image along the horizontal and vertical directions
        Rotate       Boolean          Whether to rotate the image by "Angle" deg (90deg) C.C.W
        Crop         Boolean          Whether to randomly crop the image to a box of size "width" by "height"

        num_images   number of images to read per class
        num_class    number of class

        Returns:
        ----------
        images      images array, (N, 256, 256, num_class) to be stored, here N = num_images * num_class
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Loop over all IDs and read each image in one by one

    for class_id in range(num_classes+1):
      if class_id:
        for image_id in range(num_images+1):
          if image_id:
            images.append(np.array(read_store_data('/content/drive/MyDrive/GAN/Damage prediction/Class '+ f'{class_id}/'+ f'{image_id}.txt')))

    for j in range(num_classes):
      for i in range(50):
        labels.append(j)
    
    data = np.asarray(images)
    labels = np.asarray(labels)
    labels = labels.astype(np.float64)
  
    X_train_1 = []
    y_train_1 = []
    X_train_augmented = [image for image in data]
    y_train_augmented = [label for label in labels]

    for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
        for image, label in zip(data, labels):
          if Shift:
            X_train_augmented.append(shift_image(image, dx, dy))
            y_train_augmented.append(label)
          if Flip:
            X_train_augmented.append(np.fliplr(image))
            X_train_augmented.append(np.flipud(image))
            y_train_augmented.append(label)
            y_train_augmented.append(label)
          if Rotate:
            for i in [90, 180, 270]:
              X_train_augmented.append(rotate(image, angle= i))
              y_train_augmented.append(label)

    X_train_augmented = np.array(X_train_augmented)
    y_train_augmented = np.array(y_train_augmented)
    
    if Crop:
      for image, label in zip(X_train_augmented, y_train_augmented):
        for i in range(4):
          X_train_1.append(random_crop(image, width, height))
          y_train_1.append(label)    
      X_train_1 = np.array(X_train_1)
      y_train_1 = np.array(y_train_1)
      return [X_train_1, y_train_1]  
    else:
      return [X_train_augmented, y_train_augmented]

In [ ]:
X, y = read_many_disk(0, 0, 0, 0, 6)

Next, do this to convert the grayscale data to RGB (well technically it's still grayscale, but this is done since the model architecture is built for RGB). We're bascially expanding the dimensions of the individual images and saving it back out as .jpeg files. 

In [ ]:
import imageio
for i in range(0, np.shape(X)[0], 1):
  im = np.stack((X[i],X[i], X[i]), axis = 2)
  imageio.imwrite('/content/drive/MyDrive/GAN/damage_data/'+ f'{i}'+'.jpeg', im)

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format. Depending on the resolution of your images and how many you have, this can take a while. The final dataset will be stored on Google Drive for easier access in subsequent runs.

In [ ]:
#update this to the path to your image folder
dataset_path = "/content/drive/MyDrive/GAN/damage_data/"
#give your dataset a name
dataset_name = "dam_data256"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell. All the required arguements will be set in the folling cell though.

In [ ]:
!python train.py --help

Since StyleGAN training can take forever on Colab (as per the official repo, it takes >3 days to train from scratch on 8 V100 GPUs!), we'll use a pretrained StyleGAN as the starting point for our model. Download the pre-trained .pkl file, so you can start training on top of that directly. In the case of microstructural data, you can download our trained model (available in the repo). Else, the pre-trained ffhq-1024 or horse-256 are good starting points depending on your dataset's resolution. 

In [ ]:
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-horse-config-f.pkl

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "dam_data256"
#how often should the model generate samples and a .pkl file
snapshot_count = 1
#should the images be mirrored left to right?
mirrored = False
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00045-dam_data256-11gb-gpu-bg-resumecustom/network-snapshot-000020.pkl'

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --resume={resume_from} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --augpipe="bg"

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model. 

In our case, it took about 6 hours (on a T4 GPU) to get results that were visually similar to the training dataset. 